### Packages

In [1]:
import numpy as np
exp = np.exp
array = np.array
sum = np.sum
π = np.pi
log = np.log
import numdifftools as nd

### Constants

In [2]:
kb = 1.3806e-23 # J/K
N_A = 6.0221e23 # 1/mol
R = 8.314 # J/mol-K
η = .402 # Packing fraction
v = 6.3e-5 # m^3/mol

a_ni = np.array([[0.9105631445,-0.3084016918,-0.0906148351],
                 [0.6361281449,0.1860531159,0.4527842806],
                 [2.6861347891,-2.5030047259,0.5962700728],
                 [-26.547362491,21.419793629,-1.7241829131],
                 [97.759208784,-65.255885330,-4.1302112531],
                 [-159.59154087,83.318680481,13.776631870],
                 [91.297774084,-33.746922930,-8.6728470368]])

a_ni = a_ni.T

b_ni = np.array([[0.7240946941,-0.5755498075,0.0976883116],
                 [2.2382791861,0.6995095521,-0.2557574982],
                 [-4.0025849485,3.8925673390,-9.1558561530],
                 [-21.003576815,-17.215471648,20.642075974],
                 [26.855641363,192.67226447,-38.804430052],
                 [206.55133841,-161.82646165,93.626774077],
                 [-355.60235612,-165.20769346,-29.666905585]])

b_ni = b_ni.T

In [3]:
def fx_d(T, σ, ϵ_k):
    d_ans = σ*(1-.12*exp(-3*ϵ_k/T))
    return d_ans

def fx_ρ(T, x, m, σ, η):

    k = len(x)
    d = fx_d(T, σ, ϵ_k)
    ρ_ans = 6/π*η*(sum([x[i]*m[i]*d[i]**3 for i in range(k)]))**(-1)
    return ρ_ans

def fx_ξ(x, m, d, ρ):

    k = len(x)
    ξ = array([])
    for n in range(4):
        Σ = 0
        for i in range(k):
            Σ += x[i]*m[i]*d[i]**n
        ξ = np.append(ξ, π/6*ρ*Σ)
    return ξ

def fx_g_hs_ij(x, d, ξ):

    k = len(x)
    g_hs_ij = np.zeros((k, k))
    for i in range(np.shape(g_hs_ij)[0]):
        for j in range(np.shape(g_hs_ij)[1]):
            g_hs_ij[i][j] = 1/(1 - ξ[3]) + (d[i]*d[j]/(d[i]+d[j]))*3*ξ[2]/(1-ξ[3])**2 + (d[i]*d[j]/(d[i]+d[j]))**2 * 2*ξ[2]**2/(1-ξ[3])**3

    return g_hs_ij

In [4]:
def fx_a_hc(T, x, m, σ, ϵ_k, η):

    k = len(x)
    d = fx_d(T, σ, ϵ_k)
    ρ = fx_ρ(T, x, m, σ, η)
    m̄ = sum(x*m)

    ξ = fx_ξ(x, m, d, ρ)
    g_hs_ij = fx_g_hs_ij(x, d, ξ)

    ã_hs = 1/ξ[0]*(3*ξ[1]*ξ[2]/(1-ξ[3]) + ξ[2]**3/(ξ[3]*(1-ξ[3])**2) + (ξ[2]**3/ξ[3]**2 - ξ[0])*log(1-ξ[3]))


    ã_hc = m̄*ã_hs - sum([x[i]*(m[i] - 1)*log(g_hs_ij[i][i]) for i in range(k)])

    return ã_hc

def fx_a_disp(T, x, m, σ, e_k, k_ij, η):

    k = len(x)
    ρ = fx_ρ(T, x, m, σ, η)
    m̄ = sum(x*m)


    a = a_ni[0]+ (m̄-1)/m̄*a_ni[1] + (m̄-1)/m̄*(m̄-2)/m̄*a_ni[2]
    b = b_ni[0]+ (m̄-1)/m̄*b_ni[1] + (m̄-1)/m̄*(m̄-2)/m̄*b_ni[2]



    I1 = np.round([a[i]*η**i for i in range(7)], 15)
    I2 = np.round([b[i]*η**i for i in range(7)], 15)


    I1 = np.sum(I1)
    I2 = np.sum(I2)

    σ_ij = np.zeros((k, k))
    for i in range(k):
        for j in range(k):
            σ_ij[i][j] = 1 / 2 * (σ[i] + σ[j])

    ϵ_ij = np.zeros((k, k))
    for i in range(k):
        for j in range(k):
            ϵ_ij[i][j] = (ϵ_k[i] * ϵ_k[j]) ** (1 / 2) * (1 - k_ij[i][j])

    Σ_i = 0
    for i in range(k):
        Σ_j = 0
        for j in range(k):
            Σ_j += x[i]*x[j]*m[i]*m[j]*(ϵ_ij[i][j]/T)*σ_ij[i][j]**3
        Σ_i += Σ_j
    Σ_1 = Σ_i

    Σ_i = 0
    for i in range(k):
        Σ_j = 0
        for j in range(k):
            Σ_j += x[i]*x[j]*m[i]*m[j]*(ϵ_ij[i][j]/T)**2*σ_ij[i][j]**3
        Σ_i += Σ_j
    Σ_2 = Σ_i

    C1 = (1 + m̄*(8*η - 2*η**2)/(1-η)**4 + (1-m̄)*(20*η - 27*η**2 + 12*η**3 - 2*η**4)/((1-η)*(2-η))**2)**-1


    I1 = np.round(I1,15)
    I2 = np.round(I2,15)
    C1 = np.round(C1,15)
    Σ_1 = np.round(Σ_1,15)
    Σ_2 = np.round(Σ_2,15)
    ã_disp = np.round(-2*π*ρ*I1*Σ_1 - π*ρ*m̄*C1*I2*Σ_2, 15)

#     print(I1)

#     print(a)
#     print(b)
#     print(f'I1 = {I1:.20f}')
#     print(f'I2 = {I2:.20f}')
#     print(f'C1 = {C1:.20f}')
#     print(f'Σ_1 = {Σ_1:.20f}')
#     print(f'Σ_2 = {Σ_2:.20f}')
#     print(f'a_disp = {ã_disp:.10f}')






    return ã_disp
    
def fx_a_assoc(T, x, m, σ, ϵ_k, κ_AB, ϵ_AB_k, η):

    k = len(x)
    d = fx_d(T, σ, ϵ_k)
    ρ = fx_ρ(T, x, m, σ, η)
    ξ = fx_ξ(x, m, d, ρ)
    g_hs_ij = fx_g_hs_ij(x, d, ξ)

    κ_AB_ij = np.zeros((k, k))
    for i in range(k):
        for j in range(k):
            κ_AB_ij[i][j] = (κ_AB[i] * κ_AB[j]) ** (1 / 2) * ((σ[i] * σ[j]) / (1 / 2 * (σ[i] * σ[j]))) ** 3

    ϵ_AB_ij = np.zeros((k, k))
    for i in range(k):
        for j in range(k):
            ϵ_AB_ij[i][j] = (ϵ_AB_k[1] + ϵ_AB_k[2]) / 2

    d_ij = np.zeros((k, k))
    for i in range(k):
        for j in range(k):
            d_ij[i][j] = 1/2*(d[i] + d[j])

    Δ_AB_ij = np.zeros((k, k))
    for i in range(k):
        for j in range(k):
            Δ_AB_ij[i][j] = d_ij[i][j]**3*g_hs_ij[i][j]*κ_AB_ij[i][j]*(exp(ϵ_AB_ij[i][j]/T) - 1)


    def XA_find(XA_guess, n, Δ_AB_ij, ρ, x):
        m = int(XA_guess.shape[1] / n)
        X_Ai = np.zeros_like(XA_guess)
        AB_matrix = np.asarray([[0., 1.],
                                [1., 0.]])
        Σ_2 = np.zeros((n,), dtype='float_')
        XA = np.zeros_like(XA_guess)

        for i in range(n):
            Σ_2 = 0 * Σ_2
            for j in range(n):
                Σ_2 += ρ * x[j] * (XA_guess[j, :] @ (Δ_AB_ij[i][j] * AB_matrix))
            XA[i, :] = 1 / (1 + Σ_2)

        return XA


    ncA = 2
    a_sites = 2
    iA = [1, 2]
    XA = np.zeros((ncA, a_sites), dtype='float_')

    ctr = 0
    dif = 1000.
    XA_old = np.copy(XA)
    while (ctr < 500) and (dif > 1e-9):
        ctr += 1
        XA = XA_find(XA, ncA, Δ_AB_ij, ρ, x[iA])
        dif = np.sum(abs(XA - XA_old))
        XA_old[:] = XA
    XA = XA.flatten()

    ã_assoc = 0
    for i in range(3):
        Σ_1 = 0
        for j in range(len(XA)):
            Σ_1 += log(XA[j] - 1 / 2 * XA[j] + 1 / 2)

        ã_assoc += x[i] * Σ_1

    return ã_assoc


def f_a_res(T, x, m, σ, ϵ_k, k_ij, κ_AB, ϵ_AB_k, η):

    ã_hc = fx_a_hc(T, x, m, σ, ϵ_k, η)
    ã_disp = fx_a_disp(T, x, m, σ, ϵ_k, k_ij, η)
    ã_assoc = fx_a_assoc(T, x, m, σ, ϵ_k, κ_AB, ϵ_AB_k, η)



    ã_res = ã_hc + ã_disp + ã_assoc
    # print(ã_hc, ã_disp, ã_res)

    return ã_res

def f_Z(T, x, m, σ, ϵ_k, k_ij, κ_AB, ϵ_AB_k, η):

    def g_a_res(η):
        return f_a_res(T, x, m, σ, ϵ_k, k_ij, κ_AB, ϵ_AB_k, η)

    da_dη = nd.Derivative(g_a_res)

    Z = 1 + η*da_dη(η)
    return Z

def f_h_res(T, x, m, σ, ϵ_k, k_ij, κ_AB, ϵ_AB_k, η):

    def g_a_res(T):
        return f_a_res(T, x, m, σ, ϵ_k, k_ij, κ_AB, ϵ_AB_k, η)

    da_dT = nd.Derivative(g_a_res)
    Z = f_Z(T, x, m, σ, ϵ_k, k_ij, κ_AB, ϵ_AB_k, η)

    return -T*da_dT(T) + (Z - 1)

def f_s_res(T, x, m, σ, ϵ_k, k_ij, κ_AB, ϵ_AB_k, η):

    def g_a_res(T):
        return f_a_res(T, x, m, σ, ϵ_k, k_ij, κ_AB, ϵ_AB_k, η)

    a_res = f_a_res(T, x, m, σ, ϵ_k, k_ij, κ_AB, ϵ_AB_k, η)
    Z = f_Z(T, x, m, σ, ϵ_k, k_ij, κ_AB, ϵ_AB_k, η)
    da_dT = nd.Derivative(g_a_res)

    return -T*(da_dT(T) + a_res/T) + np.log(Z)

def f_g_res(T, x, m, σ, ϵ_k, k_ij, κ_AB, ϵ_AB_k, η):

    a_res = f_a_res(T, x, m, σ, ϵ_k, k_ij, κ_AB, ϵ_AB_k, η)
    Z = f_Z(T, x, m, σ, ϵ_k, k_ij, κ_AB, ϵ_AB_k, η)

    return a_res + (Z - 1) - np.log(Z)

def f_da_dx(k, T, x, m, σ, ϵ_k, k_ij, κ_AB, ϵ_AB_k, η):

    def g_a_res(xk):
        x[k] = xk
        return f_a_res(T, x, m, σ, ϵ_k, k_ij, κ_AB, ϵ_AB_k, η)

    da_dx = nd.Derivative(g_a_res)

    return da_dx(x[k])


def f_μk_res(k, T, x, m, σ, ϵ_k, k_ij, κ_AB, ϵ_AB_k, η):

    a_res = f_a_res(T, x, m, σ, ϵ_k, k_ij, κ_AB, ϵ_AB_k, η)
    Z = f_Z(T, x, m, σ, ϵ_k, k_ij, κ_AB, ϵ_AB_k, η)

    def f_da_dx(j):

        def g_a_res(xj):
            x[j-1] = xj
            return f_a_res(T, x, m, σ, ϵ_k, k_ij, κ_AB, ϵ_AB_k, η)

        da_dk = nd.Derivative(g_a_res)

        return da_dk(x[j-1])

    # print(f_da_dx(1), f_da_dx(2), f_da_dx(3))

    Σ = 0
    for j in range(len(x)):
        Σ += x[j]*f_da_dx(j+1)

    return (a_res + (Z - 1) + f_da_dx(k) - Σ)*kb*T

def f_lnΦ(k, T, x, m, σ, ϵ_k, k_ij, κ_AB, ϵ_AB_k, η):

    μk_res = f_μk_res(k, T, x, m, σ, ϵ_k, k_ij, κ_AB, ϵ_AB_k, η)
    Z = f_Z(T, x, m, σ, ϵ_k, k_ij, κ_AB, ϵ_AB_k, η)

    return μk_res/kb/T - np.log(Z)



In [5]:
T = 233.15
x = array([.1, .3, .6]) # Mole fraction
m = array([1, 1.6069, 2.0020]) # Number of segments
σ = array([3.7039, 3.5206, 3.6184]) # Temperature-Independent segment diameter σ_i (Aᵒ)
ϵ_k = array([150.03, 191.42, 208.11]) # Depth of pair potential / Boltzmann constant (K)
k_ij = array([[0.00E+00,3.00E-04,1.15E-02],
              [3.00E-04,0.00E+00,5.10E-03],
              [1.15E-02,5.10E-03,0.00E+00]])
κ_AB = array([0, 0, 0])
ϵ_AB_k = array([0, 0, 0])
η = .402

δ = .0001
h = δ*x[0]

print(f'{f_a_res(T, x, m, σ, ϵ_k, k_ij, κ_AB, ϵ_AB_k, η):.30f}')

x1 = np.array([0.09998000,0.30000000,0.60000000])
x2 = np.array([0.09999000,0.30000000,0.60000000])
x3 = np.array([0.10001000,0.30000000,0.60000000])
x4 = np.array([0.10002000,0.30000000,0.60000000])

a1 = f_a_res(T, x1, m, σ, ϵ_k, k_ij, κ_AB, ϵ_AB_k, η)
a2 = f_a_res(T, x2, m, σ, ϵ_k, k_ij, κ_AB, ϵ_AB_k, η)
a3 = f_a_res(T, x3, m, σ, ϵ_k, k_ij, κ_AB, ϵ_AB_k, η)
a4 = f_a_res(T, x4, m, σ, ϵ_k, k_ij, κ_AB, ϵ_AB_k, η)

ans = (a1 - 8*a2 + 8*a3 - a4)/(12*h)

print(a1 - a2)
print(a2 - a3)
print(a3 - a4)
print(ans)

-3.573711925238200137755484320223
-1.0623621600203137e-06
-2.124577663131788e-06
-1.0622154826833707e-06
0.10622888331202061
